In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np


# 2. Load dataset (use your processed/train split)
df = pd.read_csv("/Users/riadanas/Desktop/housing regression MLE/data/processed/feature_engineered.csv")

pd.set_option('display.max_columns', None)  # Show all columns in the output
pd.set_option('display.max_rows', None)     # Show all rows in the output

In [2]:
print(df.shape)
df.head(2)

(734668, 41)


,year,quarter,month,median_sale_price,median_list_price,median_ppsf,median_list_ppsf,homes_sold,pending_sales,new_listings,inventory,median_dom,avg_sale_to_list,sold_above_list,off_market_in_two_weeks,bank,bus,hospital,mall,park,restaurant,school,station,supermarket,Total Population,Median Age,Per Capita Income,Total Families Below Poverty,Total Housing Units,Median Rent,Median Home Value,Total Labor Force,Unemployed Population,Total School Age Population,Total School Enrollment,Median Commute Time,price,lat,lng,zipcode_freq,city_encoded
0,2012,1,3,46550.0,217450.0,31.813674,110.183666,14.0,23.0,44.0,64.0,59.5,0.943662,0.142857,0.043478,12.0,2.0,4.0,1.0,60.0,45.0,57.0,4.0,7.0,5811.0,36.3,33052.0,5811.0,2677.0,710.0,279500.0,3171.0,460.0,5408.0,5408.0,2492.0,200773.999557,33.7338,-84.3922,118,210734.122857
1,2012,1,3,200000.0,7500.0,104.931794,79.265873,1.0,1.0,1.0,2.0,290.0,0.909091,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,2441.0,41.8,20241.0,2385.0,1108.0,641.0,94600.0,1171.0,52.0,2376.0,2376.0,1018.0,105863.681174,40.4744,-79.8632,118,137187.780044


In [3]:
high_vif_features = [
    "Total School Enrollment",
    "Total School Age Population",
    "Total Population",
    "Total Families Below Poverty",
    "Total Labor Force",
    "Median Commute Time"
]

df.drop(columns=high_vif_features, inplace=True)

In [4]:
print(df.shape)
df.head(2)

(734668, 35)


,year,quarter,month,median_sale_price,median_list_price,median_ppsf,median_list_ppsf,homes_sold,pending_sales,new_listings,inventory,median_dom,avg_sale_to_list,sold_above_list,off_market_in_two_weeks,bank,bus,hospital,mall,park,restaurant,school,station,supermarket,Median Age,Per Capita Income,Total Housing Units,Median Rent,Median Home Value,Unemployed Population,price,lat,lng,zipcode_freq,city_encoded
0,2012,1,3,46550.0,217450.0,31.813674,110.183666,14.0,23.0,44.0,64.0,59.5,0.943662,0.142857,0.043478,12.0,2.0,4.0,1.0,60.0,45.0,57.0,4.0,7.0,36.3,33052.0,2677.0,710.0,279500.0,460.0,200773.999557,33.7338,-84.3922,118,210734.122857
1,2012,1,3,200000.0,7500.0,104.931794,79.265873,1.0,1.0,1.0,2.0,290.0,0.909091,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,41.8,20241.0,1108.0,641.0,94600.0,52.0,105863.681174,40.4744,-79.8632,118,137187.780044


In [5]:
# Define target and features
target = "price"   # <-- adjust if needed
X = df.drop(columns=[target])
y = df[target]

# Train/validation split (80/20)
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("Train shape:", X_train.shape)
print("Validation shape:", X_val.shape)

Train shape: (587734, 34)
Validation shape: (146934, 34)


In [6]:
# ================================================
# Standardization (Z-score scaling)
# ================================================
scaler = StandardScaler()

# Fit on training data, transform both train & val
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

print("Scaled data shape:", X_train_scaled.shape)

Scaled data shape: (587734, 34)


In [7]:
X_train_scaled

array([[ 0.85412972,  0.42823685,  0.12241959, ...,  0.45717295,
         0.        , -0.9589783 ],
       [ 1.55916612,  0.42823685,  0.12241959, ..., -0.364987  ,
         0.        , -0.70924289],
       [ 1.20664792, -1.37313415, -1.04780612, ...,  0.17428409,
         0.        , -0.68745836],
       ...,
       [-1.26097948,  1.32892236,  1.58520174, ...,  0.79972413,
         0.        ,  0.36835821],
       [ 1.55916612, -1.37313415, -1.34036255, ...,  0.62037806,
         0.        , -0.26755396],
       [-1.26097948,  1.32892236,  1.00008888, ...,  0.65884228,
         0.        , -1.18401905]])

In [8]:
# ================================================
# Linear Regression (no regularization)
# ================================================
lr = LinearRegression()
lr.fit(X_train_scaled, y_train)

y_pred_lr = lr.predict(X_val_scaled)

print("Linear Regression performance:")
print("MAE:", mean_absolute_error(y_val, y_pred_lr))
print("RMSE:", np.sqrt(mean_squared_error(y_val, y_pred_lr)))
print("R²:", r2_score(y_val, y_pred_lr))


Linear Regression performance:
MAE: 43046.055373057265
RMSE: 106335.90421068388
R²: 0.8905650482949187


- This difference leads to Lasso performing feature selection by potentially setting some coefficients to zero, 
- whereas Ridge regression shrinks all coefficients towards zero but doesn't eliminate any

In [9]:
# ================================================
# Ridge Regression (L2 regularization) --> Shrinks but doesn't eliminate features
# ================================================
ridge = Ridge(alpha=1.0)  # alpha controls regularization strength
ridge.fit(X_train_scaled, y_train)

y_pred_ridge = ridge.predict(X_val_scaled)

print("Ridge Regression performance:")
print("MAE:", mean_absolute_error(y_val, y_pred_ridge))
print("RMSE:", np.sqrt(mean_squared_error(y_val, y_pred_ridge)))
print("R²:", r2_score(y_val, y_pred_ridge))


Ridge Regression performance:
MAE: 43046.03199828546
RMSE: 106336.04445728724
R²: 0.8905647596268437


In [10]:
# ================================================
# Lasso Regression (L1 regularization). --> can eliminate some features
# ================================================
lasso = Lasso(alpha=0.1)  # alpha controls regularization strength
lasso.fit(X_train_scaled, y_train)

y_pred_lasso = lasso.predict(X_val_scaled)

print("Lasso Regression performance:")
print("MAE:", mean_absolute_error(y_val, y_pred_lasso))
print("RMSE:", np.sqrt(mean_squared_error(y_val, y_pred_lasso)))
print("R²:", r2_score(y_val, y_pred_lasso))

Lasso Regression performance:
MAE: 43045.92270693916
RMSE: 106335.9073969237
R²: 0.8905650417367197


In [11]:
# ================================================
# ElasticNet (L1 + L2 regularization) --> Combination of both
# ================================================
elastic = ElasticNet(alpha=0.1, l1_ratio=0.5)  
# l1_ratio=0.5 → equal mix of L1 and L2
elastic.fit(X_train_scaled, y_train)

y_pred_elastic = elastic.predict(X_val_scaled)

print("ElasticNet performance:")
print("MAE:", mean_absolute_error(y_val, y_pred_elastic))
print("RMSE:", np.sqrt(mean_squared_error(y_val, y_pred_elastic)))
print("R²:", r2_score(y_val, y_pred_elastic))


ElasticNet performance:
MAE: 43140.786690710134
RMSE: 110670.80341936779
R²: 0.8814607106444009


- Scaling worked → all models gave stable results.

- Linear Regression is a strong baseline → regularization didn’t add much.

- Collinearity exists but isn’t catastrophic → dropping features / ratios may help, but Ridge didn’t really need to fix it.

- Multicollinearity does not hurt predictive accuracy much → the model was already using redundant features that all point in the same direction.

- Dropping them removed redundancy, but the predictive signal remained, so performance didn’t collapse.